### Implementation of the paper https://link.springer.com/article/10.1007/s13226-023-00452-9 "Encryption and decryption of signed graph matrices through RSA algorithm"

### Sender Side

In [40]:
"""
  This class is the base class for creating SignedGraphs. THe class.add_edge can be used to add a new edge from a source vertex to a destination vertix
  If no edge type is defined, it will be assumed as a positive edge
"""
class SignedGraph:
    def __init__(self, num_vertices):
        self.num_vertices = num_vertices
        self.adj_matrix = [[0] * num_vertices for _ in range(num_vertices)]

    def add_edge(self, source, destination, edge_type=1):
        if source >= self.num_vertices or destination >= self.num_vertices:
            raise IndexError("Vertex index out of range")
        if edge_type != 1 and edge_type!= -1:
            raise IndexError("Edge Type Should Either be 1 or -1")
        if source == destination:
            raise IndexError("You can not add a loop edge")
        self.adj_matrix[source][destination] = edge_type
        self.adj_matrix[destination][source] = edge_type

    def get_adjacency_matrix(self):
        return self.adj_matrix

# Example usage
NUM_VERTICES = 10
graph = SignedGraph(NUM_VERTICES)
graph.add_edge(0, 1)
graph.add_edge(1, 2)
graph.add_edge(2, 3, -1)
graph.add_edge(3,4, 1)
graph.add_edge(4,7, -1)
graph.add_edge(1,6,1)
graph.add_edge(9,8, -1)
graph.add_edge(0,9)


adj_matrix = graph.get_adjacency_matrix()
for row in adj_matrix:
    print(row)

[0, 1, 0, 0, 0, 0, 0, 0, 0, 1]
[1, 0, 1, 0, 0, 0, 1, 0, 0, 0]
[0, 1, 0, -1, 0, 0, 0, 0, 0, 0]
[0, 0, -1, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, -1, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, -1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, -1]
[1, 0, 0, 0, 0, 0, 0, 0, -1, 0]


In [2]:
"""
  THe following function changes a symmetric signed matrix to an upper triangular matrix
  Essentially ripping off the the lower triangular matrix and the diagonal elements.
"""

def convert_to_upper_triangular(adjacency_matrix, number_of_vertices):
    for i in range(number_of_vertices):
        for j in range(number_of_vertices):
            if j<i:
                adjacency_matrix[i][j] = 0
    upper_triangular_matrix = [adjacency_matrix[i][i+1:] for i in range(number_of_vertices)]
    return upper_triangular_matrix

In [3]:
upper_triangular_matrix = convert_to_upper_triangular(adj_matrix, NUM_VERTICES)
upper_triangular_matrix

[[1, 0, 0, 0, 0, 0, 0, 0, 1],
 [1, 0, 0, 0, 1, 0, 0, 0],
 [-1, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0],
 [0, 0, -1, 0, 0],
 [0, 0, 0, 0],
 [0, 0, 0],
 [0, 0],
 [-1],
 []]

In [4]:
"""
This function converts an upper triangular matrix to a list.
It will add elements starting from the last row of the matrix from right to left

"""
def convert_utm_to_list(upper_triangular_matrix):
    list_of_digits = []
    for i in range(NUM_VERTICES-1, -1, -1):
        for j in range(len(upper_triangular_matrix[i])-1, -1, -1):
            list_of_digits.append(upper_triangular_matrix[i][j])
    return list_of_digits
            # print(f'Round {i} {j}')

In [5]:
list_of_digits = convert_utm_to_list(upper_triangular_matrix)

In [6]:
list_of_digits

[-1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 -1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 -1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1]

In [7]:
len(list_of_digits)

45

In [8]:
"""
Convert the list of digits to strings of 01s and -1s as defined in the paper.
It will start from left scan the string of 0's, 1's and -1's.
Add -1 as a seperate string when encountered and 0s and 1s between -1s as one string

"""

def convert_list_to_strings(input_list):
    strings = []
    current_string = ""

    for value in input_list:
        if value == -1:
            if current_string:
                strings.append(current_string)
                current_string = ""
            strings.append(str(value))
        else:
            current_string += str(value)

    if current_string:
        strings.append(current_string)

    return strings


In [9]:
strings_of_digits = convert_list_to_strings(list_of_digits)

In [10]:
strings_of_digits

['-1', '00000000000', '-1', '00000001000000', '-1', '00010001100000001']

In [11]:
"""
THis funciton adds a 1 at the start and end of any string that is not -1
"""
def wrap_ones_around_strings(strings_of_digits):
    appended_strings_of_digits = []
    for s in strings_of_digits:
        if int(s) != -1:
            s = "1" + s + "1"
        appended_strings_of_digits.append(s)
    return appended_strings_of_digits

In [12]:
appended_strings_of_digits = wrap_ones_around_strings(strings_of_digits)

In [13]:
appended_strings_of_digits

['-1', '1000000000001', '-1', '1000000010000001', '-1', '1000100011000000011']

In [14]:
"""
Change the binary strings to their decimal counter parts

"""
def binary_to_decimal(binary_list):
    decimal_list=[]
    for i in binary_list:
        decimal_list.append(str(int(i, base=2)))
    return decimal_list

In [15]:
decimalized_numbers = binary_to_decimal(appended_strings_of_digits)

In [16]:
decimalized_numbers

['-1', '4097', '-1', '32897', '-1', '280067']

In [17]:
decimalized_numbers


['-1', '4097', '-1', '32897', '-1', '280067']

In [18]:
import rsa

In [19]:

publicKey, privateKey = rsa.newkeys(512)

In [20]:
"""
Use private key of rsa to encrypt the numbers
"""
encStrings = []
for i in decimalized_numbers:
    encMessage = rsa.encrypt(i.encode(),publicKey)
    # print(encMessage)
    encStrings.append(encMessage )

In [21]:
encStrings

[b'\x0b\x91@\x9bX\xb5\x8a\x88\xdd08\x9bX\x82\x15\x1eC\x06S\xe2%\x97e\x98K\xeb*)X\x8frv \x0c\x95~I\xc0\xe5R\xa5\x05f\xc1H\xb2\xdf\x0b4:{I\xef\xb1\x06\xf2\\\xd0=\xfb12f\x87',
 b'\x1aW5O\x9c\xca\xf8sl\x9au\xd5X+A\\\x1fK\xd8s\\c\xd6\xdb_S\x03\xe2\x1f\xe5\xc5\x83\x04\xd5\xaf$S\xcaO\x14\xdd\x94K\x85+\xd3\xca\xbd\xa4S@i\x8e\x18FC\xde\x15Y\xa9\x1cj\r\xc2',
 b'\x84\xf5\x9b\xd6q\xca\xdcp\xc2\xadb\xeb\xfd9\xebCs\x96\xfc\x82\xb4\x8b\xb5\xd85L\x83\x9d4\xb5\xe4\n\x16\xb3<\xdc\xf8\x94\x9d\x17\xa5\xe2\x03\x0blU\xb2\x94J0J\xaa|\x9a\xaaj\x91e\xc7m\xf2\x96\xb1\xc8',
 b'4\xf4\xfd~b\xb9?\xc9\x95\x06\xcd\x1a\xb3:\xb8_\xef \x01\x82\xfcm\x15n\x0c\xab\xa4\\\xac\x138\xe13\x11\xddv\x83$\xfd\x94\xfc\x9bm\xc7\x02\xeb\x0b\xab\xd7\x16\x9ey\xc3\x0f\x8c`\xd2\xd4\x0f4\xf3}\x8d\xd6',
 b'o\xcd!\xe1\x8c]\x9b\x8b\xa8\xffAA\xee\xea\xa4\xcb:\xb7W\xdalg\xeb{M\xa83\n\x0c\xda\x06\xfd\xceS\xa2\xbd\xf1\xa2=\x92\x18\xb7\xe4\x95-"\x8a\xc5\x81[\x1b\xd5\xaf\x94\x86\x03\x86\xb9g\xc6Xa\x9b\xa3',
 b"3u <\x14{{\xa5\xf9\xc3\x01\xc9\xc9\x9

### Receiver Side

#### The next lines of code will be carried out in the receiver side

In [22]:
"""
Decrpyt the message using private key
"""
decStrings=[]
for i in encStrings:
    decMessage = rsa.decrypt(i, privateKey).decode()
    decStrings.append(decMessage)

In [23]:
decStrings

['-1', '4097', '-1', '32897', '-1', '280067']

In [25]:
"""
Convert Decimals to binary
"""
decoded_binary = []
for i in decStrings:
    num = str(bin(int(i)).replace("0b", ""))
    decoded_binary.append(num)

In [26]:
decoded_binary

['-1', '1000000000001', '-1', '1000000010000001', '-1', '1000100011000000011']

In [27]:
"""
Strip the 1s at the start and end of strings that are not -1 and then make a list from the strings
"""
decoded_binary_list = []
for i in decoded_binary:
    if i != "-1":
        bin_string = i[1:-1]
        for j in bin_string:
            decoded_binary_list.append(int(j))
    else:
        decoded_binary_list.append(int(i))

00000000000
00000001000000
00010001100000001


In [28]:
decoded_binary_list

[-1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 -1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 -1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1]

In [29]:
len(decoded_binary_list)

45

In [30]:
len(decoded_binary_list) == len(list_of_digits)

True

In [31]:
"""

Convert the list back to upper triangular matrix.
It should be the same way that we constructed the list. Start from last row and add elements up to but not including diagonal positions from right to left
"""

def convert_list_to_utm(list_binary):
    k=9
    utm = []
    for i in range(NUM_VERTICES):
        temp_list = []
        for j in range(k):
            temp_list.append(decoded_binary_list.pop())
        utm.append(temp_list)
        k-=1
    return utm
            # print(f'Round {i} {j}')

In [32]:
constructed_utm = convert_list_to_utm(decoded_binary_list)

In [33]:
constructed_utm

[[1, 0, 0, 0, 0, 0, 0, 0, 1],
 [1, 0, 0, 0, 1, 0, 0, 0],
 [-1, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0],
 [0, 0, -1, 0, 0],
 [0, 0, 0, 0],
 [0, 0, 0],
 [0, 0],
 [-1],
 []]

In [34]:
"""
Add zeros to the matrix to make an adjacency matrix
"""
def add_zeros_to_utm(utm, number_of_vertices):
    k=1
    for i in range(number_of_vertices):
        for j in range(k):
            utm[i].insert(0, 0)
        k+=1
    return utm

In [35]:
utm_with_zeros = add_zeros_to_utm(constructed_utm, NUM_VERTICES)

In [36]:
utm_with_zeros

[[0, 1, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 1, 0, 0, 0, 1, 0, 0, 0],
 [0, 0, 0, -1, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, -1, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, -1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

In [37]:
"""
Make a complete adjacency matrix. By adding 1s and -1s to the mirror positions of where they are
"""

for i in range(NUM_VERTICES):
    for j in range(i):
        if(utm_with_zeros[j][i] != 0):
            utm_with_zeros[i][j] = utm_with_zeros[j][i]

In [38]:
utm_with_zeros

[[0, 1, 0, 0, 0, 0, 0, 0, 0, 1],
 [1, 0, 1, 0, 0, 0, 1, 0, 0, 0],
 [0, 1, 0, -1, 0, 0, 0, 0, 0, 0],
 [0, 0, -1, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, -1, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, -1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, -1],
 [1, 0, 0, 0, 0, 0, 0, 0, -1, 0]]